In [ ]:
import cv2
import mediapipe as mp
from ultralytics import YOLO

model_yolo = YOLO('best.pt') 

mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

cap = cv2.VideoCapture(0)

while True:
    
    ret, frame = cap.read()

    results_yolo = model_yolo(frame) 

    if isinstance(results_yolo, list) and results_yolo:

        results_yolo = results_yolo[0]  

    if results_yolo.boxes is not None:
        for det in results_yolo.boxes.data.numpy(): 
            label = int(det[4])
            confidence = det[5]
            bbox = det[:4]

            cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), (0, 255, 0), 2)
            cv2.putText(frame, f'{model_yolo.names[label]}: {confidence:.2f}', (int(bbox[0]), int(bbox[1]) - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results_pose = pose.process(frame_rgb)

    if results_pose.pose_landmarks:
        for landmark in results_pose.pose_landmarks.landmark:
            h, w, _ = frame.shape
            cx, cy = int(landmark.x * w), int(landmark.y * h)
            cv2.circle(frame, (cx, cy), 5, (255, 0, 0), -1)

    cv2.imshow('Real-time Monitoring', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()